In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import label_binarize
from sklearn import svm
from modules.stats import * 
from modules.PCA import * 
from modules.LDA import * 
from modules.read_data import *
from modules.ROC import *

In [ ]:
path = '../../Assets/Image Dataset'
data_class = ['coast', 'forest', 'highway', 'mountain', 'opencountry']

C = [] #36*23
D = []

for i in data_class:
	C.append(read_ID(path, i, 'train').flatten().reshape(-1, 36*23))
	D.append(read_ID(path, i, 'dev').flatten().reshape(-1, 36*23))

In [ ]:
C_y = []
D_y = []

for i in range(5):
	C_y.append(i * np.ones(C[i].shape[0]))
	D_y.append(i * np.ones(D[i].shape[0]))

In [ ]:
train = np.concatenate(C, axis=0)
dev = np.concatenate(D, axis=0)

X = (train - train.min(axis=0))/(train.max(axis=0) - train.min(axis=0))
X_d = (dev - dev.min(axis=0))/(dev.max(axis=0) - dev.min(axis=0))
Y = np.concatenate(C_y, axis=0)
Y_d = np.concatenate(D_y, axis=0)

In [ ]:
pca = False
lda = False
lda = ~pca & lda #if pca is true then lda is false

In [ ]:
if pca:
	pc = 25
	μ, Σ = stats(X)
	eigval, Q = PCA(Σ, pc)
	X = X @ Q
	X_d = X_d @ Q
if lda:
	pc = 25
	μ, Σ = stats(X)
	μk = []
	for c in range(5):
		μk.append(stats(X[Y == c])[0])
	μk = np.array(μk)
	v = LDA(X,Y,μ,μk,pc)
	X = np.abs(X @ v)
	X_d = np.abs(X_d @ v)

In [ ]:
clf = svm.SVC(probability=True)
clf.fit(X, Y)

In [ ]:
y_pred = clf.predict(X_d)
score = clf.predict_proba(X_d)

In [ ]:
cm = confusion_matrix(Y_d, y_pred)
accuracy = accuracy_score(Y_d,y_pred)
ax = sns.heatmap(cm, annot = True, cmap ='plasma',linecolor ='black', linewidths = 1, fmt = '.0f', xticklabels = data_class, yticklabels = data_class)
ax.set_title(f'Accuracy = {accuracy}')

In [ ]:
TPR = 0; FPR = 0
for c in range(5):
	tpr, fpr = ROC(label_binarize(Y_d, classes=[0,1,2,3,4])[:,c], score[:,c])
	TPR += tpr/5
	FPR += fpr/5
plt.plot(FPR, TPR)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title("ROC")